In [12]:
ROOT = Path('/media/anton/b8150e49-6ff0-467b-ad66-40347e8bb188/anton/BACHELOR')

Printing all Drugbank data in .txt files

In [11]:
import xml.etree.ElementTree as etree
import xml
l = []
t=[]
l0 = []
l1 = []
l2 = []
k = 0
lst = []
f = False
N = 20
g = False
name = "None"
path_to_file = str(ROOT / 'full database.xml')
for event, elem in etree.iterparse(path_to_file):
    #print(elem.tag)
   # pass
    if elem.text != None:
        #print(xml.etree.ElementTree.tostringlist(elem))
        #if elem.tag.split("{http://www.drugbank.ca}")[1] not in ("citation", "resource"):
        if f:
            lst.append(elem.tag.split("{http://www.drugbank.ca}")[1] + "|" + elem.text)
            if elem.tag.split("{http://www.drugbank.ca}")[1] == "id" and len(elem.text) >= 1:
                g = True
        if (elem.tag == "{http://www.drugbank.ca}drug") and (k <= N):
            f = True
            
            k += 1
            print("!")
            #print(lst)
            if g:
                with open(str(ROOT) + str(k) + '.txt', 'w') as f:
                    f.write("")
                with open(str(ROOT) + str(k) + '.txt', 'a') as f:
                    for item in lst:
                        f.write("|%s|\n" % item.encode('utf8'))
            g = False
            lst = []
        if k >= N:
            break
    elem.clear()

!
!
!
!
!
!
!
!
!
!
!
!
!
!
!
!
!
!
!
!


In [10]:
import pandas as pd
import os

df = pd.DataFrame({"unii" : l})
df.to_csv(str(ROOT /"drugbank_unii.csv"), index=False)

# Get some files from Drugbank

In [ ]:
# Drugbank authentication data
NAME = 'maksimov.as@phystech.edu'
PASSWORD = "0511drugsandbanks"

In [5]:
import requests
import os
import zipfile

def get_from_drugbank(FOLDER_DB, url_file):
    response = requests.get(url_file, auth=(NAME, PASSWORD))
    path_to_file =  os.path.join(FOLDER_DB, url_file.split('/')[-1])
    print("Downloading " + url_file.split("/")[-1])
    open(path_to_file, 'wb').write(response.content)

    with zipfile.ZipFile(path_to_file, 'r') as zip_ref:
        print("Extracting " + url_file.split("/")[-1])
        zip_ref.extractall(FOLDER_DB)
    return 0



FOLDER_DB = str(ROOT / 'trash')

url_file = 'https://www.drugbank.ca/releases/5-1-3/downloads/all-drug-links'

get_from_drugbank(FOLDER_DB, url_file)

Extracting all-drug-links


0

using password

In [15]:
#cURL command

import subprocess
from pathlib import Path

def cmd(command):
    subprocess.check_output(command, shell=True)

NAME = 'your name' #'maksimov.as@phystech.edu'#
PASSWORD = 'your password' #0511....
URL = 'https://www.drugbank.ca/releases/5-1-3/downloads/target-all-polypeptide-ids'
OUT_FILE = Path(ROOT / 'Drugbank')
OUT_FILE = OUT_FILE / (URL.split('/')[-1] +'.zip')

cmd('curl -Lfv -o ' + str(OUT_FILE) + ' -u ' + NAME + ':' + PASSWORD + ' ' + URL)

# XML database parsing and saving needed fields

In [6]:
import xml.etree.ElementTree as etree
import xml

def db_tag(element, string):
    return element.tag.split("{http://www.drugbank.ca}")[1] == string

source = "/media/anton/b8150e49-6ff0-467b-ad66-40347e8bb188/anton/BACHELOR/full database.xml"
# get an iterable
context = etree.iterparse(source, events=("start", "end"))

# turn it into an iterator
context = iter(context)

# get the root element
event, root = next(context)
# for Python 2: event, root = context.next()

k = 0
all_ligands_unii = []
all_ligands_db_id = [] # DB ids
all_ligands_name = [] # usual names
all_ligands_ids = [] # list of lists of ids in different DBs
all_resources_of_ligands = [] # list of lists of resources in different DBs
all_ids_of_targets = [] # list of lists of lists of ids in different DBs
all_resources_of_targets = [] # list of lists of lists of resources in different DBs
unii = ""
f_approved = False
kol = 0
for event, elem in context:

    if event == "end" and db_tag(elem,"drug"): #and k == 1:
        k += 1
        
        #refresh all lists of ids and resources
        ids_l = []
        resources_l = []
        ids_t = []
        resources_t = []
        ids_target = []
        resources_target = []
        db_id = []
                            
        f_resource = True #drug - True, target - False
        
        #get unii
        for item in list(elem):           
            if db_tag(item, "unii"):
                unii = item.text
            if db_tag(item, "drugbank-id"):
                db_id.append(item.text)
            if db_tag(item, "name"):
                name = item.text
                #print("\n")
            #checking if approved by FDA
            if db_tag(item, "products"):
                for it1 in list(item):
                    for it in list(it1):
                        if db_tag(it, "approved"):
                            if it.text == "true":
                                f_approved = True
                                
            #identifiers of drug                    
            if db_tag(item, "external-identifiers"):
                kol += 1
                
                for it1 in list(item):
                    for it in list(it1):
                        if db_tag(it, "resource"):
                            resources_l.append(it.text)

                        if db_tag(it, "identifier"):
                            id_l = it.text
                            ids_l.append(id_l)
                            
            #identifiers of targets
            if db_tag(item, "targets"):
                for it1 in list(item):
                    for it2 in list(it1):
                        if db_tag(it2, "polypeptide"):
                            for it3 in list(it2):
                                #print(it3.tag)
                                if db_tag(it3, "external-identifiers"):
                                    for it4 in list(it3):
                                        for it5 in list(it4):
                                            #print("!!")
                                            if db_tag(it5, "resource"):
                                                #print(it.tag + "|" + it.text)
                                                resources_t.append(it5.text)
                                            if db_tag(it5, "identifier"):
                                                #print(it.text)
                                                ids_t.append(it5.text)
                                    ids_target.append(ids_t)
                                    ids_t = []
                                    resources_target.append(resources_t)
                                    resources_t = []
        root.clear()
        
    #if approved => add information        
    if unii != "" and f_approved:
        f_approved = False
        all_ligands_name.append(name)
        all_ligands_unii.append(unii)
        all_ligands_ids.append(ids_l)
        all_ligands_db_id.append(db_id)
        ids_l = []
        all_resources_of_ligands.append(resources_l)
        resources_l = []

        all_ids_of_targets.append(ids_target)
        ids_t = []
        all_resources_of_targets.append(resources_target)
        resources_target = []

## dump all collected information

In [19]:
import os
import json
from pathlib import Path

def make_dir(dirList):
    for dirName in dirList:
        if not os.path.exists(dirName):
            os.mkdir(dirName)
            print("Directory " , dirName ,  " Created ")
        else:
            pass
        
ROOT = Path('/media/anton/b8150e49-6ff0-467b-ad66-40347e8bb188/anton/BACHELOR/')

names = ['all_ligands_name', 'all_ligands_unii', 'all_ligands_ids', 'all_ligands_db_id', 'all_resources_of_ligands',
         'all_ids_of_targets', 'all_resources_of_targets']
name_full = str(ROOT / 'Drugbank_extracted')
make_dir([name_full])
for name in names:
    with open(str(Path(name_full) / (name + ".txt")), 'w') as f:
        exec('json.dump(' + name + ', f, ensure_ascii=False)')

# load all information, if didn't upload it before

In [40]:
import os
import json
from pathlib import Path

names = ['all_ligands_name', 'all_ligands_unii', 'all_ligands_ids', 'all_ligands_db_id', 'all_resources_of_ligands',
         'all_ids_of_targets', 'all_resources_of_targets']
name_full = str(ROOT / 'Drugbank_extracted')

for name in names:
    with open(str(Path(name_full) / (name + ".txt")), 'r') as f:
        exec(name + '= json.load(f)')

Checking databases of ligands

In [ ]:
print(len(all_resources_of_ligands))
k = 0
b = []
for i in all_resources_of_ligands:
    f = False
    for j in i:
        if j == 'PubChem Substance':
            f = True
    if f:
        k += 1
    else:
        for el in i:
            b.append(el)
print(set(b))
from collections import Counter

print(Counter(b).values())
types = dict(zip(b, Counter(b).values()))
print(types)
print(sum(Counter(b).values()))
print(k, 'from ', len(all_resources_of_ligands), 'have PubCHEM ID')

# Retrieve PubCHEM ids of ligands and get their SMILES

https://pubchempy.readthedocs.io/en/latest/guide/install.html

In [38]:
import pubchempy as pcp

pubchem_ids = []
pubchem_smiles = []
pubchem_numbers = []
for ind1, i in enumerate(all_resources_of_ligands):
    if ind1 == 5: #-1 for normal work
        break
    else:
        for ind2, j in enumerate(i):
            if j == 'PubChem Substance':
                try:
                    pubchem = all_ligands_ids[ind1][ind2]
                    c = pcp.Compound.from_cid(pubchem)
                    pubchem_smiles.append(c.isomeric_smiles)
                    pubchem_numbers.append(ind1)  
                    pubchem_ids.append(pubchem) 
                except:
                    pass            
                break

additional manual check of id name smiles

In [39]:
for ind, i in enumerate(pubchem_numbers):
    print(ind)
    print(pubchem_ids[ind], all_ligands_name[pubchem_numbers[ind]], pubchem_smiles[ind])

0
46507011 Lepirudin COC1=CC=CC(=C1)CN2C(=O)C3=C(C(=CN3)C4=CC=CC=C4)NC2=S
1
46507042 Cetuximab CCOC1=CC=C(C=C1)NS(=O)(=O)C2=CC(=NN2)C(=O)NC3=CC(=CC=C3)SC
2
46507792 Dornase alfa CCOC(=O)C1=C(N(N=C1)C2=NC=C(C=C2)C(=O)NC3=CC=C(C=C3)C(F)(F)F)N
3
46506950 Denileukin diftitox C1=CC=C(C=C1)C2=CNC3=C2NC(=S)N(C3=O)CCC(=O)O
4
46506732 Etanercept CC1=C(C=C(C=C1)NC(=O)CN2C(=CC(=O)N3C2=CC(=N3)C)C)C


get smiles by pubchem for manual check

In [ ]:
pubchem =
c = pcp.Compound.from_cid(pubchem)
print(c.isomeric_smiles)

# Get pdbs with similar substructures as input SMILES

In [108]:
import requests
import os
import xml.etree.ElementTree as ET

pubchem = 46507011
smiles = 'CCOC1=CC=C(C=C1)NS(=O)(=O)C2=CC(=NN2)C(=O)NC3=CC(=CC=C3)SC' #'OC(=O)c1ccc(OCc2ccccc2)cc1'#

 #'\#'http://www.rcsb.org/pdb/rest/smilesQuery?smiles=OC(=O)c1ccc(OCc2ccccc2)cc1&search_type=similarity&similarity=0.7'


path = ROOT / 'SMILES'
make_dir([str(path)])

# trying to find appropriate similarity level to find at least one structure just by descending (mb, implement binary search?)
step = 0.05
sim = 0.40 + step
pdbs_from_smiles = []
while not pdbs_from_smiles:
    url = 'http://www.rcsb.org/pdb/rest/smilesQuery?smiles=' + smiles \
    + '&search_type=similarity&similarity=' + str(sim)
    pdbs_from_smiles = []
    sim -= step
    r = requests.get(url, allow_redirects=True)
    a = open(os.path.join(path, str(pubchem) + '.xml'), 'wb').write(r.content)

    tree = ET.parse(os.path.join(path, str(pubchem) + '.xml'))
    root = tree.getroot()
    for child in root:
        for child1 in child:
            #print(child1.tag, child1.attrib)
            pdbs_from_smiles.append(child1.attrib['structureId'])
print(pdbs_from_smiles)
print(sim)

['1FLS', '1FM1', '1GFW', '1IH0', '1KV2', '1KZN', '1PZK', '1TBF', '1UDT', '1UHO', '1UVT', '1XOS', '1XOT', '1XP0', '2AM1', '2BHH', '2BU6', '2CNH', '2EA4', '2FZZ', '2H42', '2I0A', '2I0D', '2IEJ', '2IL2', '2J7T', '2O8H', '2P16', '2VIP', '2VTI', '2W3L', '2WEL', '2WF1', '2WF3', '2WF4', '2X9E', '2XBA', '3B2R', '3BSA', '3C6T', '3C7Q', '3CDB', '3CS7', '3CTQ', '3CWJ', '3DLG', '3DM2', '3DM6', '3DMJ', '3DOK', '3DOL', '3DY6', '3FZS', '3GI3', '3GI4', '3GI5', '3GI6', '3GWT', '3HA8', '3HHK', '3HKC', '3HV6', '3IL5', '3IUB', '3IUE', '3IW5', '3JWQ', '3KI1', '3KQB', '3KQC', '3M35', '3M36', '3M37', '3MXD', '3NPC', '3PEQ', '3QCJ', '3QRI', '3QRJ', '3SHY', '3SHZ', '3SOS', '3T8M', '3TWD', '3UUZ', '3V94', '3V9B', '3VHV', '3W1F', '3WIY', '3WIZ', '3ZM5', '3ZM6', '4A2Z', '4AA7', '4AAW', '4AC3', '4AZ0', '4BTI', '4CKU', '4DX0', '4DYB', '4E1Z', '4EBW', '4EP9', '4EPH', '4FE6', '4FKJ', '4G2W', '4G2Y', '4I31', '4I32', '4I33', '4I4E', '4I4F', '4JVG', '4KN4', '4KN7', '4LP6', '4LXB', '4LXD', '4LYW', '4MUI', '4NG9', '4NMH',

In [110]:
uniprot = 'P00533'
path = ROOT / 'SMILES'
make_dir([str(path)])
List = open(str(ROOT / 'hive' / 'pdb' / uniprot / (uniprot + "_pdbs.txt"))).readlines()
for el in List:
    if el.rstrip() in pdbs_from_smiles:
        print(el)

5HIB



In [ ]:
#print(all_resources_of_ligands[ligands_unii.index("EFY6W0M8TG")+1:ligands_unii.index("EFY6W0M8TG")+3])
#print(all_ids_of_ligands[ligands_unii.index("EFY6W0M8TG")+1:ligands_unii.index("EFY6W0M8TG")+3])

How many different targets?

In [60]:
a = []
#print(all_ids_of_targets)
for i in all_ids_of_targets:
    for j in i:
        if j not in a:
            a.append(j)
len(a)

2513

# Further goes more or less obsolete code

## Check ligands and targets from table Drugbank-PDB

In [8]:
import pandas as pd
db_pdb = pd.read_csv("/media/anton/b8150e49-6ff0-467b-ad66-40347e8bb188/anton/BACHELOR/Drugbank/drugTable.csv", sep=',', lineterminator='\n', header=0)
#print(db_pdb["DrugBank ID"].values.tolist())

In [20]:
print(len(set(db_pdb["Generic Name"].values.tolist())))

508


In [13]:
found = []
not_found = []
flat_ids = [item for sublist in all_ligands_db_id for item in sublist]
for i in db_pdb["DrugBank ID"].values.tolist():
    if i in flat_ids:
        found.append(i)
    else:
        not_found.append(i)

In [17]:
print(flat_ids)

['DB00001', 'BTD00024', 'BIOD00024', 'DB00002', 'BTD00071', 'BIOD00071', 'DB00003', 'BTD00001', 'BIOD00001', 'DB00004', 'BTD00084', 'BIOD00084', 'DB00005', 'BTD00052', 'BIOD00052', 'DB00006', 'BTD00076', 'EXPT03302', 'BIOD00076', 'DB02351', 'DB00007', 'BTD00009', 'BIOD00009', 'DB00008', 'BTD00043', 'BIOD00043', 'DB00009', 'BTD00050', 'BIOD00050', 'DB00011', 'BTD00096', 'BIOD00096', 'DB00084', 'DB00012', 'BTD00032', 'BIOD00032', 'DB00013', 'BTD00030', 'BIOD00030', 'DB00014', 'BTD00113', 'BIOD00113', 'DB00015', 'BTD00013', 'BIOD00013', 'DB00016', 'BTD00103', 'BIOD00103', 'DB08923', 'DB00017', 'BTD00025', 'BIOD00025', 'DB00018', 'BTD00023', 'BIOD00023', 'DB00019', 'BTD00094', 'BIOD00094', 'DB00020', 'BTD00035', 'BIOD00035', 'DB00022', 'BTD00048', 'BIOD00048', 'DB00023', 'BTD00011', 'BIOD00011', 'DB00024', 'BTD00020', 'BIOD00020', 'DB00025', 'BTD00029', 'BIOD00029', 'DB13162', 'DBSALT002413', 'DB00026', 'BTD00060', 'BIOD00060', 'DB00027', 'BTD00036', 'BIOD00036', 'DB00028', 'BTD00091', 'BI

In [15]:
print(len(set(not_found)))
print(len(set(found)))
print(set(not_found))

58
450
{'DB04786', 'DB01244', 'DB04552', 'DB01361', 'DB00914', 'DB01094', 'DB01051', 'DB04942', 'DB11699', 'DB01412', 'DB00147', 'DB00274', 'DB00336', 'DB04729', 'DB04794', 'DB00311', 'DB01628', 'DB00170', 'DB00212', 'DB01021', 'DB00179', 'DB01055', 'DB08800', 'DB03313', 'DB00778', 'DB00154', 'DB03575', 'DB01092', 'DB00197', 'DB08954', 'DB00786', 'DB00127', 'DB00148', 'DB01245', 'DB04787', 'DB03849', 'DB01283', 'DB04133', 'DB01103', 'DB08915', 'DB06693', 'DB01422', 'DB00866', 'DB01603', 'DB01296', 'DB04570', 'DB01454', 'DB00234', 'DB05266', 'DB04743', 'DB01614', 'DB01336', 'DB01536', 'DB01034', 'DB08838', 'DB00821', 'DB00606', 'DB04573'}


In [16]:
print(db_pdb["DrugBank ID"].values.tolist())
#DB01245

['DB01454', 'DB01454', 'DB01454', 'DB01536', 'DB01536', 'DB01048', 'DB05812', 'DB00284', 'DB00284', 'DB00284', 'DB00284', 'DB01614', 'DB01614', 'DB01614', 'DB01614', 'DB00316', 'DB00316', 'DB00819', 'DB00819', 'DB00819', 'DB00819', 'DB00819', 'DB00819', 'DB00551', 'DB06151', 'DB06151', 'DB00945', 'DB00945', 'DB00787', 'DB00787', 'DB00787', 'DB00640', 'DB00640', 'DB00640', 'DB00640', 'DB08916', 'DB08916', 'DB08916', 'DB08838', 'DB08838', 'DB08838', 'DB08838', 'DB08838', 'DB08838', 'DB08838', 'DB11363', 'DB08915', 'DB08915', 'DB00630', 'DB09026', 'DB00523', 'DB00523', 'DB00523', 'DB00523', 'DB00523', 'DB00523', 'DB06203', 'DB00132', 'DB00132', 'DB00404', 'DB00866', 'DB00866', 'DB00866', 'DB00770', 'DB00770', 'DB00915', 'DB00915', 'DB00915', 'DB00479', 'DB00594', 'DB00594', 'DB00594', 'DB00594', 'DB00513', 'DB00513', 'DB01118', 'DB01118', 'DB01118', 'DB00321', 'DB00321', 'DB00321', 'DB00381', 'DB00381', 'DB00381', 'DB00381', 'DB00381', 'DB00182', 'DB00182', 'DB00182', 'DB00182', 'DB00182'

# Another table

In [27]:
import pandas as pd
db_uniprot = pd.read_csv("/media/anton/b8150e49-6ff0-467b-ad66-40347e8bb188/anton/BACHELOR/Drugbank/uniprot links.csv", sep=',', lineterminator='\n', header=0)

In [29]:
print(all_ids_of_targets)

[[['HGNC:3535', 'F2', 'M17262', '339641', '2362', 'P00734', 'THRB_HUMAN']], [['HGNC:3236', 'EGFR', 'X00588', '757924', '1797', 'P00533', 'EGFR_HUMAN'], ['HGNC:3620', 'FCGR3B', 'X16863', '31322', 'O75015', 'FCG3B_HUMAN'], ['HGNC:1246', 'C1R', 'X04701', '29539', '2334', 'P00736', 'C1R_HUMAN'], ['HGNC:1241', 'C1QA', 'AF135157', '4894854', 'P02745', 'C1QA_HUMAN'], ['HGNC:1242', 'C1QB', 'X03084', '573114', 'P02746', 'C1QB_HUMAN'], ['HGNC:1245', 'C1QC', 'AF087892', '33150626', 'P02747', 'C1QC_HUMAN'], ['HGNC:3619', 'FCGR3A', 'X52645', '31324', 'P08637', 'FCG3A_HUMAN'], ['HGNC:1247', 'C1S', 'X06596', '763110', '2335', 'P09871', 'C1S_HUMAN'], ['HGNC:3613', 'FCGR1A', 'X14356', '31332', 'P12314', 'FCGR1_HUMAN'], ['HGNC:3616', 'FCGR2A', 'M31932', '182474', 'P12318', 'FCG2A_HUMAN'], ['HGNC:3618', 'FCGR2B', 'U87560', '4099445', 'P31994', 'FCG2B_HUMAN'], ['HGNC:15626', 'FCGR2C', 'X17652', '32074', 'P31995', 'FCG2C_HUMAN']], [], [['HGNC:6008', 'IL2RA', 'X01057', '33813', '1695', 'P01589', 'IL2RA_HUMA

In [35]:
#print(db_uniprot["UniProt ID"].values.tolist())
found = []
not_found = []
flat_ids = [item  for sublist in all_ids_of_targets for subsublist in sublist for item in subsublist]
for i in db_uniprot["UniProt ID"].values.tolist():
    if i in flat_ids:
        found.append(i)
    else:
        not_found.append(i)

In [36]:
print(len(set(not_found)))
print(len(set(found)))
print(set(not_found))

192
2502
{'Q16775', 'O15440', 'Q9L5C8', 'Q8N0U8', 'O75715', 'P11759', 'Q9P2R7', 'Q07820', 'P16050', 'P18283', 'P06435', 'P36894', 'P05181', 'Q9BZV3', 'P19440', 'P54802', 'P0A3R9', 'Q96I59', 'Q8VP84', 'P77390', 'P10635', 'P12532', 'Q9NUB1', 'Q8TE23', 'O60513', 'Q08257', 'Q82122', 'P36275', 'P04936', 'Q27796', 'Q16678', 'P20815', 'Q99643', 'P16070', 'Q07817', 'P08870', 'Q9ZEU2', 'P02929', 'A1L3X4', 'Q9Y3I0', 'Q9H4Y5', 'Q9BUP3', 'P06434', 'O14880', 'Q96I99', 'Q14872', 'Q8WWT9', 'O43776', 'Q9NR19', 'P35754', 'P02943', 'Q9BZW2', 'P09467', 'P51606', 'Q9UBX3', 'Q16873', 'P68133', 'O95865', 'P0A953', 'Q59976', 'Q92945', 'P0A094', 'P31153', 'P61889', 'Q86VB7', 'O60512', 'Q00266', 'Q9HAB3', 'A0A1Q8GFY7', 'P55055', 'Q9UJ70', 'P53597', 'P0A0K8', 'Q13183', 'O95881', 'P10620', 'P04732', 'P22891', 'P59796', 'Q9I194', 'P98066', 'Q6TGC4', 'P49228', 'P17540', 'Q9RVD6', 'P48029', 'P22340', 'P55263', 'P98155', 'Q13133', 'P83689', 'P11413', 'P46439', 'P10915', 'Q8N339', 'O60909', 'Q7L5Y9', 'P11712', 'Q9NS1

In [5]:
#for ID in all_ligands_db_ids:
import numpy as np
import requests
import os
home = "/media/anton/b8150e49-6ff0-467b-ad66-40347e8bb188/anton/BACHELOR"
ID = 'DB00509'
ligand_pdbs = []
percent = []
#target_pdbs
for i in range(3):
    frame = db_pdb.loc[db_pdb['DrugBank ID'] == ID]["PDB ID " + str(i+1)].dropna()
    for el in frame:
        ligand_pdbs.append(el)
print(ligand_pdbs)
ligand_pdbs = set(ligand_pdbs)
path_id = os.path.join(home, "Drugbank", ID)
try:  
    os.mkdir(path_id)
except OSError:  
    pass#print ("Directory %s failed (maybe exists)" % fda_folder)
for pdb in ligand_pdbs:
    url_file = "https://files.rcsb.org/download/" + pdb + ".pdb"

    r = requests.get(url_file, allow_redirects=True)
    path =  os.path.join(path_id, url_file.split("/")[-1])
    print("Downloading " + url_file.split("/")[-1])
    open(path, 'wb').write(r.content)

['1Y0X', '4LNX', '4LNX', '1Y0X']


# some old tests (garbage)

In [17]:
#exec("%s = %d" % (x,2))
def get_json(name):
    with open('/media/anton/b8150e49-6ff0-467b-ad66-40347e8bb188/anton/BACHELOR/' + name + ".txt", 'r') as f: 
        a = json.load(f)
    return a

#print(get_json("all_ids_of_targets"))
a = get_json("all_resources_of_targets")
k = 0
uniprot = 0
for el in a:
    for el1 in el:
        k += 1
        for el2 in el1:
            if el2 == u'UniProtKB':
                uniprot += 1

In [18]:
print(k, uniprot)

(9048, 9048)


In [7]:
print(len(I))
kol = 0
for sp in I:
    kol += sp.count("ChEMBL")
print(kol)

11922
6754


In [9]:
import pandas as pd
a = pd.read_csv("/media/anton/b8150e49-6ff0-467b-ad66-40347e8bb188/anton/FDA/Data/unii_inchikey.csv", sep=',', lineterminator='\n', header=0)

In [28]:
c = 0
c1 = 0
All = []
processed = []
for elem in drugs1["unii"].values.tolist():
    if elem in targets:
        c += 1
        ind = targets.index(elem)
        if elem not in processed:
            processed.append(elem)
            for el in I[ind]:
                if "ChEMBL" not in I[ind]:
                    All.append(el)
        if "ChEMBL" in I[ind]:
            c1 += 1
from collections import Counter
print(All.count("ChEMBL"))
print(Counter(All).values())
types = dict(zip(All, Counter(All).values()))
print(types)
print(sum(Counter(All).values()))
print(c)
print(c1)
#print(len(processed))

0
[16, 1, 868, 40, 514, 2, 9, 11, 29, 345, 61, 105, 53, 23, 2, 44]
{'UniProtKB': 2, 'PubChem Substance': 105, 'PharmGKB': 23, 'Wikipedia': 29, 'GenBank': 53, 'KEGG Compound': 868, 'Drugs Product Database (DPD)': 44, 'Therapeutic Targets Database': 11, 'KEGG Drug': 40}
2123
3090
1986


In [30]:
len(a["unii"].values.tolist())

1793

In [21]:
a["unii"].values.tolist()

1793

In [1]:
import xml.etree.ElementTree as etree
import xml

def db_tag(element, string):
    return element.tag.split("{http://www.drugbank.ca}")[1] == string

source = "/media/anton/b8150e49-6ff0-467b-ad66-40347e8bb188/anton/BACHELOR/full database.xml"
# get an iterable
context = etree.iterparse(source, events=("start", "end"))

# turn it into an iterator
context = iter(context)

# get the root element
event, root = context.next()
drugs1 = []
k = 0
for event, elem in context:
    if event == "end" and db_tag(elem,"drug"): #and k == 1:
        f = True
        for item in list(elem):
            if db_tag(item, "unii"):
                unii = item.text
                if item.text in drugs1:
                    f = False
            if db_tag(item, "products"):
                for it1 in list(item):
                    for it in list(it1):
                        if db_tag(it, "approved"):
                            #print(it.tag + "|" + it.text)
                            if it.text == "true" and f:
                                k += 1
                                drugs1.append(unii)
                                f = False
                           # numbers.append(it)
        root.clear()

In [13]:
len(a)

1793

In [17]:
print(drugs1)

            unii
0     Y43GF64R34
1     PQX0D8J21J
2     953A26OA1Y
3     25E79B5CTM
4     OP401G7OJC
5     TN9BEX005G
6     EFY6W0M8TG
7     Q46947FE7K
8     1RXS4UE564
9     41697D4Z5C
10    15UQ94PT4P
11    83G67E21XI
12    0F65R8P09N
13    DQA630RIE9
14    64FS3BFH5W
15    7SFC6U2VI5
16    47BPR3V3MP
17    3A58010674
18    5TAA004E22
19    G8RGG88B68
20    G4FQ3CKY5R
21    AVX3D5A4LM
22    P89DR4NY54
23    9013DUQ28K
24    5IE62321P4
25    66Y330CJHS
26    5O8V541HJ6
27    1Y17CTI5SR
28    WGD229O42W
29    5Y9QQM372Q
...          ...
3061  QHD0E51VON
3062  W4B15Z80I6
3063  3Y5612490S
3064  2J092A2Q42
3065  46Z3N8299R
3066  08MVT1AQQ9
3067  JJ3895I54I
3068  8C5Y219Z75
3069  752SY38R6C
3070  5GMR90S4KN
3071  8F049NKY49
3072  TM54L796SN
3073  S883J9JDYX
3074  W8N8R55022
3075  87PZU03K0K
3076  29J08234MX
3077  RS6978S7U6
3078  72CM5JF07H
3079  420U24CBE7
3080  BY4TSK952Y
3081  252S9L5606
3082  C67221CB9Q
3083  6QVL057INT
3084  8WB9PV3YW5
3085  9J6GJ0890E
3086  4SV59689SK
3087  9R3D3Y0U

In [25]:
len(drugs1)

3091

In [8]:
len(set(drugs))

7612

In [7]:
import pandas as pd
import os
df = pd.DataFrame({"unii" : drugs1})
df.to_csv(os.path.join("/media/anton/b8150e49-6ff0-467b-ad66-40347e8bb188/anton/BACHELOR/Drugbank", "drugbank_unii_fda.csv"), index=False)

In [3]:
import pandas as pd
a = pd.read_csv("/media/anton/b8150e49-6ff0-467b-ad66-40347e8bb188/anton/FDA/Data/unii_inchikey.csv", sep=',', lineterminator='\n', header=0)

In [4]:
drugs1 = pd.read_csv("/media/anton/b8150e49-6ff0-467b-ad66-40347e8bb188/anton/BACHELOR/Drugbank/drugbank_unii_fda.csv", sep=',', lineterminator='\n', header=0)

In [38]:
b = []
for el in drugs1['unii'].to_list():
    if el not in a["unii"].to_list():
        b.append(el)

In [19]:
len(b)

2252

In [20]:
print(b)

['Y43GF64R34', 'PQX0D8J21J', '953A26OA1Y', '25E79B5CTM', 'OP401G7OJC', 'TN9BEX005G', 'EFY6W0M8TG', 'Q46947FE7K', '1RXS4UE564', '41697D4Z5C', '15UQ94PT4P', '83G67E21XI', '0F65R8P09N', 'DQA630RIE9', '7SFC6U2VI5', '47BPR3V3MP', '5TAA004E22', 'G8RGG88B68', 'G4FQ3CKY5R', 'AVX3D5A4LM', 'P89DR4NY54', '5IE62321P4', '66Y330CJHS', '5O8V541HJ6', 'WGD229O42W', '5Y9QQM372Q', '21K6M2I7AG', '47RRR83SK7', 'ENR1LLB0FP', 'AC71R787OV', 'HM5641GA6F', 'QMS40680K6', '76LA80IG2G', 'M89N0Q7EQR', '0Y70779M1F', '2P471X1Z11', '3JGY52XJNA', '9X7O8V25IT', '08GY9K1EUO', 'OON1HFZ4BA', 'FYS6T7F842', 'NQX9KB6PCL', 'Q6U6J48BWY', 'X85G7936GV', 'JGH8MYC891', '8GZG754X6M', 'F43I396OIS', '7D96IR0PPM', 'XRO4566Q4R', 'HW3H7D91F6', 'ZIF514RVZR', 'B72HH48FLU', '076WHW89TW', 'Y87Y826H08', 'TTD90R31WZ', '56588OP40D', '64R4OHP8T0', 'AVT680JB39', 'P188ANX8CK', '4F4X42SYQ6', '9927MT646M', 'JGA39ICE2V', 'YB12NQZ1YN', '4Q52C550XK', '0343IGH41U', 'N824AOU5XV', 'E211KPY694', 'FQ3DRG0N5K', '8X1OXL3SNU', '3A189DH42V', '27T56C7KK0', 'BDO5

In [39]:
c = []
for el in a["unii"].to_list():
    if el not in drugs1['unii'].to_list():
        c.append(el)

In [22]:
len(c)

954

In [23]:
print(c)

['E7199S1YWR', '786Z46389E', 'B53E3NMY5C', '207LT9J9OC', '3ST302B24A', '3M8608UQ61', '5U85DBW7LO', '5968Y6H45M', '6GNT3Y5LMF', 'BK76465IHM', 'OLH94387TE', '9N7R477WCK', '00FN6IH15D', '3I3T11UD2S', 'FD171B778Y', '9O25354EPJ', '86P6PQK0MU', '146T0TU1JB', 'UTI8907Y6X', 'F446C597KA', '804826J2HU', 'W1792A2RVD', 'W5S57Y3A5L', 'MC34P30298', 'G447S0T1VC', 'Z49QDT0J8Z', '19XTS3T51U', '33067B3N2M', '48A5M73Z4Q', '57WVB6I2W0', '9TN87S3A3C', 'Q2WXR1I0PK', '0EWG668W17', '0K2I505OTV', 'J8BDZ68989', '6871619Q5X', '08I79HTP27', '0020414E5U', 'TC2D6JAD40', '864V2Q084H', '2UY4M2U3RA', 'CQ903KQA31', 'QTG126297Q', '83MVU38M7Q', 'U1O3J18SFL', '0VE05JYS2P', '6E7SN358SE', '4A3O49NGEZ', '4BA73M5E37', 'BW9B0ZE037', '6E8ZO8LRNM', 'N1SN99T69T', '021SEF3731', 'TH25PD4CCB', 'L4D5UA6CB4', 'W76I6XXF06', '77TI35393C', 'T20OQ1YN1W', 'JTE4MNN1MD', '9J765S329G', '50MD4SB4AP', '2S3PL1B6UJ', '11SV1951MR', 'WY6W63843K', '26LUD4JO9K', 'OBN7UDS42Y', '10X8X4P12Z', '644VL95AO6', '36H71644EQ', 'R6DXU4WAY9', 'X2ELS050D8', '9IFA

In [40]:
import pandas as pd
import os
df = pd.Series(c)
print(c)
df.to_csv(os.path.join("/media/anton/b8150e49-6ff0-467b-ad66-40347e8bb188/anton/BACHELOR/Drugbank", "drugbank_not_in_fda.csv"), index=False)

['E7199S1YWR', '786Z46389E', 'B53E3NMY5C', '207LT9J9OC', '3ST302B24A', '3M8608UQ61', '5U85DBW7LO', '5968Y6H45M', '6GNT3Y5LMF', 'BK76465IHM', 'OLH94387TE', '9N7R477WCK', '00FN6IH15D', '3I3T11UD2S', 'FD171B778Y', '9O25354EPJ', '86P6PQK0MU', '146T0TU1JB', 'UTI8907Y6X', 'F446C597KA', '804826J2HU', 'W1792A2RVD', 'W5S57Y3A5L', 'MC34P30298', 'G447S0T1VC', 'Z49QDT0J8Z', '19XTS3T51U', '33067B3N2M', '48A5M73Z4Q', '57WVB6I2W0', '9TN87S3A3C', 'Q2WXR1I0PK', '0EWG668W17', '0K2I505OTV', 'J8BDZ68989', '6871619Q5X', '08I79HTP27', '0020414E5U', 'TC2D6JAD40', '864V2Q084H', '2UY4M2U3RA', 'CQ903KQA31', 'QTG126297Q', '83MVU38M7Q', 'U1O3J18SFL', '0VE05JYS2P', '6E7SN358SE', '4A3O49NGEZ', '4BA73M5E37', 'BW9B0ZE037', '6E8ZO8LRNM', 'N1SN99T69T', '021SEF3731', 'TH25PD4CCB', 'L4D5UA6CB4', 'W76I6XXF06', '77TI35393C', 'T20OQ1YN1W', 'JTE4MNN1MD', '9J765S329G', '50MD4SB4AP', '2S3PL1B6UJ', '11SV1951MR', 'WY6W63843K', '26LUD4JO9K', 'OBN7UDS42Y', '10X8X4P12Z', '644VL95AO6', '36H71644EQ', 'R6DXU4WAY9', 'X2ELS050D8', '9IFA

/media/anton/b8150e49-6ff0-467b-ad66-40347e8bb188/anton/anaconda3/envs/py37/lib/python2.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """


In [5]:
print(drugs1)

            unii
0     Y43GF64R34
1     PQX0D8J21J
2     953A26OA1Y
3     25E79B5CTM
4     OP401G7OJC
5     TN9BEX005G
6     EFY6W0M8TG
7     Q46947FE7K
8     1RXS4UE564
9     41697D4Z5C
10    15UQ94PT4P
11    83G67E21XI
12    0F65R8P09N
13    DQA630RIE9
14    64FS3BFH5W
15    7SFC6U2VI5
16    47BPR3V3MP
17    3A58010674
18    5TAA004E22
19    G8RGG88B68
20    G4FQ3CKY5R
21    AVX3D5A4LM
22    P89DR4NY54
23    9013DUQ28K
24    5IE62321P4
25    66Y330CJHS
26    5O8V541HJ6
27    1Y17CTI5SR
28    WGD229O42W
29    5Y9QQM372Q
...          ...
3061  QHD0E51VON
3062  W4B15Z80I6
3063  3Y5612490S
3064  2J092A2Q42
3065  46Z3N8299R
3066  08MVT1AQQ9
3067  JJ3895I54I
3068  8C5Y219Z75
3069  752SY38R6C
3070  5GMR90S4KN
3071  8F049NKY49
3072  TM54L796SN
3073  S883J9JDYX
3074  W8N8R55022
3075  87PZU03K0K
3076  29J08234MX
3077  RS6978S7U6
3078  72CM5JF07H
3079  420U24CBE7
3080  BY4TSK952Y
3081  252S9L5606
3082  C67221CB9Q
3083  6QVL057INT
3084  8WB9PV3YW5
3085  9J6GJ0890E
3086  4SV59689SK
3087  9R3D3Y0U